In [ ]:
import numpy as np
from scipy.interpolate import griddata
import matplotlib.pyplot as plt

In [ ]:
DG_PG_5  = -7
DG_PG_WT = -11
DG_PE_5  = -2
DG_PE_WT = -9

In [ ]:
N = 1000
xmin = 1e-12
xmax = .9

gr_x = np.linspace(xmin, xmax, N)
gr_y = np.linspace(xmin, xmax, N)
grid_x, grid_y = np.meshgrid(gr_x, gr_y)

PG = grid_x
PE = grid_y
PC =  np.full((N, N), 1.0) - (PG + PE)

RT = 0.6
KCG5  = np.exp(-DG_PG_5 / RT)
KCGWT = np.exp(-DG_PG_WT / RT)
KCE5  = np.exp(-DG_PE_5 / RT)
KCEWT = np.exp(-DG_PE_WT / RT)

data = (PC + KCG5 * PG + KCE5 * PE) / (PC + KCGWT * PG + KCEWT * PE)
data = np.log(data)

In [ ]:
%matplotlib notebook
plt.contourf(np.log10(grid_x), np.log10(grid_y), data, levels=100)
# plt.contour(np.log10(grid_x), np.log10(grid_y), data, levels=40, color='black')
plt.xlabel("PG content")
plt.ylabel("PE content")
plt.title("Relative log-probability of ELIC5 conformation")
# plt.xscale("log")
# plt.yscale("log")
plt.colorbar()

In [ ]:
%matplotlib notebook

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
ax.plot_surface(np.log10(grid_x), np.log10(grid_y), data)


# Ternary PG Titration

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import scipy.constants as const

In [ ]:
dGs = pd.read_csv('freeEnergies.csv', comment="#", header=[0], index_col=[0,1])
dGs

In [ ]:
nVals = 48
temp=300
#RT = temp*const.R/(1000*const.calorie)
RT = 0.59
terTitr = pd.DataFrame()
terTitr['NPG'] = np.ones(nVals)
tmp = list([0.75])
tmp.extend(1.5**np.arange(nVals-1))
terTitr['NPC'] = tmp
terTitr['NPE'] = np.divide(terTitr.NPC, 2)
structures = np.array(['WT', 'ELIC5'])
headgroups = np.array(['PG', 'PC', 'PE'])

In [ ]:
terTitr['logPG'] = np.log10(terTitr.NPG/np.sum(terTitr.iloc[:,0:3], axis=1), )

for struct in structures:
    for alpha in headgroups:
        for beta in headgroups:
            if beta != alpha:
                div = np.divide(terTitr[f'N{alpha}'], terTitr[f'N{beta}'])
                exp =  np.exp(-dGs.loc[(f'PO{alpha}',beta), struct]/RT)
                terTitr[struct,f'p{alpha}_p{beta}'] = div*exp

In [ ]:
for struct in structures:
    for alpha in headgroups:
        for beta in headgroups:
            if beta != alpha:
                gamma = headgroups[np.logical_and(headgroups!=beta, headgroups!=alpha)]
                gamma = gamma[0]
                denom = terTitr[struct, f'p{alpha}_p{beta}']
                numer = terTitr[struct, f'p{beta}_p{gamma}']
                print(f'{struct}{alpha}')
                terTitr[struct, f'fp{alpha[-1]}'] = 1/(1+1/denom+numer/denom)

In [ ]:
terTitr['WT', 'fpC']